# Modelos de clasificación

En esta segunda parte del Test 1 de clasificación, nos centraremos en elegir los modelos que se adapten mejor al tipo de datos que hemos preparado en el primer script EDA, analizar sus resultados y justificar la elección en función de métricas clave de rendimiento.

Para obtener unos resultados óptimos en este problema de clasificación sin incurrir en tiempos de procesamiento desmedidos, probaremos a implementar los siguientes modelos:

· Regresión logística: Intentaremos obtener una referencia inicial de accuracy, precision, ROC-AUC y recall. Es un modelo rápido de entrenar y sencillo de interpretar, por lo que nos dará una idea inicial del comportamiento de los datos.

· Ramdom Forest: Entrenaremos un conjunto de árboles de decisión con muestras aleatorias del Dataset para intentar obtener una clasificación más profunda de los datos.

· Gradient Boosting (XGBoost): Intentaremos maximizar la precisión (accuracy) utilizando este modelo que combina el entrenamiento secuencial de árboles de decisión con modelos más fuertes.

## Preparación de los datos

· One-hot encoding para las variables categóricas

· Separación en datos de entrenamiento y test

· Estandarización de las variables numéricas

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [19]:
# Cargamos el Dataset limpio
df = pd.read_csv('dataset_EDA.csv')

# Definimos características de la variable objetivo
X = df.drop(['symboling'], axis=1)
y = df['symboling']

# Definimos las columnas categóricas y numéricas para el preprocesamiento
categorical_columns = ['make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 
                       'drive-wheels', 'engine-location', 'horsepower_binned', 'price_binned']

numeric_columns = ['normalized-losses', 'wheel-base', 'length', 'width', 'height', 
                   'curb-weight', 'engine-size', 'bore', 'stroke', 'compression-ratio', 
                   'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'power_to_weight', 'make_price_ratio']

# Dividimos el conjunto de datos en entrenamiento (80%) y test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creamos un transformador para aplicar One-hot encoding a las variables categóricas y estandarización a las numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ]
)

In [20]:
# Creamos el pipeline de preprocesamiento
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

## Entrenamiento y evaluación de los modelos

### Regresión logística

In [21]:
# Entrenamos el modelo de regresión logística
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_preprocessed, y_train)

# Predicciones
y_pred_logreg = logreg.predict(X_test_preprocessed)

# Métricas de evaluación
print("Regresión Logística:")
print(classification_report(y_test, y_pred_logreg))
print("ROC-AUC:", roc_auc_score(y_test, logreg.predict_proba(X_test_preprocessed), multi_class='ovr'))

Regresión Logística:
              precision    recall  f1-score   support

          -2       1.00      1.00      1.00         1
          -1       0.83      1.00      0.91         5
           0       0.87      0.72      0.79        18
           1       0.56      0.62      0.59         8
           2       0.50      0.60      0.55         5
           3       0.75      0.75      0.75         4

    accuracy                           0.73        41
   macro avg       0.75      0.78      0.76        41
weighted avg       0.75      0.73      0.74        41

ROC-AUC: 0.9216106422628162


### Random Forest

In [22]:
# Entrenamos el modelo Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_preprocessed, y_train)

# Predicciones
y_pred_rf = rf.predict(X_test_preprocessed)

# Métricas de evaluación
print("Random Forest:")
print(classification_report(y_test, y_pred_rf))
print("ROC-AUC:", roc_auc_score(y_test, rf.predict_proba(X_test_preprocessed), multi_class='ovr'))

Random Forest:
              precision    recall  f1-score   support

          -2       1.00      1.00      1.00         1
          -1       0.83      1.00      0.91         5
           0       0.93      0.72      0.81        18
           1       0.54      0.88      0.67         8
           2       1.00      0.40      0.57         5
           3       0.80      1.00      0.89         4

    accuracy                           0.78        41
   macro avg       0.85      0.83      0.81        41
weighted avg       0.84      0.78      0.78        41

ROC-AUC: 0.9774502269067487


### XGBoost

In [23]:
# Entrenamos el modelo XGBoost
xgboost_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgboost_model.fit(X_train_preprocessed, y_train)

# Predicciones
y_pred_xgb = xgboost_model.predict(X_test_preprocessed)

# Métricas de evaluación
print("XGBoost:")
print(classification_report(y_test, y_pred_xgb))
print("ROC-AUC:", roc_auc_score(y_test, xgboost_model.predict_proba(X_test_preprocessed), multi_class='ovr'))

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [-2 -1  0  1  2  3]

In [ ]:
"""SUCIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO!!!!!!!!!!!!!!

# 1. One-hot encoding para variables categóricas
categoric_columns = df.select_dtypes(include=['object']).columns
df = pd.get_dummies(df, columns=categoric_columns)

# Definir las características (X) y la variable objetivo (y)
X = df.drop('symboling', axis=1)  # Consideramos "symboling" como la variable objetivo
y = df['symboling']

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Estandarización de los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 3. Entrenamiento y evaluación de los modelos de clasificación

# Modelo 1: Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Evaluar el modelo Random Forest
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest Report:\n", classification_report(y_test, y_pred_rf))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))

# Modelo 2: Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

# Evaluar el modelo Logistic Regression
print("\nLogistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Logistic Regression Report:\n", classification_report(y_test, y_pred_lr))
print("Logistic Regression Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))

# Modelo 3: Support Vector Machine (SVM)
svm = SVC()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

# Evaluar el modelo SVM
print("\nSVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("SVM Report:\n", classification_report(y_test, y_pred_svm))
print("SVM Confusion Matrix:\n", confusion_matrix(y_test, y_pred_svm))
"""